In [1]:
from pathlib import Path
import torch
import json
import pandas as pd
import tqdm

from au2v.config import ModelConfig, TrainerConfig
from au2v.dataset_manager import load_dataset_manager
from au2v.trainer import PyTorchTrainer
from au2v.model import load_model, PyTorchModel

In [2]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    hit_counts = {k: 0 for k in top_k}
    for seq_index, context_items, target_indices in tqdm.tqdm(test_dataset):
        rec_list = model.output_rec_lists(
            seq_index=torch.LongTensor([seq_index]),
            item_indices=torch.LongTensor([context_items]),
            cand_item_indices=torch.arange(num_item),
            k=max(top_k),
        )
        for k in top_k:
            hit_counts[k] += len(set(target_indices) & set(rec_list[0][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [3]:
run_configs = [
    {
        "name": "User2Vec (d=16)",
        "model_name": "doc2vec",
        "d_model": 16,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "User2Vec (d=32)",
        "model_name": "doc2vec",
        "d_model": 32,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "User2Vec (d=64)",
        "model_name": "doc2vec",
        "d_model": 64,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    # {
    #     "name": "AU2V (d=16)",
    #     "model_name": "attentive",
    #     "d_model": 16,
    #     "epochs": 10,
    # },
    # {
    #     "name": "AU2V (d=32)",
    #     "model_name": "attentive",
    #     "d_model": 32,
    #     "epochs": 10,
    # },
    # {
    #     "name": "AU2V (d=64)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 10,
    # },
    # {
    #     "name": "AU2V (wo weight-tying)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 10,
    #     "use_weight_tying": False,
    #     "use_attention": True,
    #     "use_meta": True,
    # },
    # {
    #     "name": "AU2V (wo attention)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 10,
    #     "use_weight_tying": True,
    #     "use_attention": False,
    #     "use_meta": True,
    # },
    # {
    #     "name": "AU2V (wo meta)",
    #     "model_name": "attentive",
    #     "d_model": 64,
    #     "epochs": 10,
    #     "use_weight_tying": True,
    #     "use_attention": True,
    #     "use_meta": False,
    # },
]

In [4]:
results = {}

for run_config in run_configs:
    results[run_config["name"]] = {"accuracy": []}
    model_config = ModelConfig(
        weight_decay=1e-8,
        max_embedding_norm=1,
        d_model=run_config["d_model"],
        lr=5e-5,
        use_attention=run_config["use_attention"],
        use_meta=run_config["use_meta"],
        use_weight_tying=run_config["use_weight_tying"],
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        model_name=run_config["model_name"],
        load_dataset=False,
        save_dataset=False,
        load_model=False,
        ignore_saved_model=True,
        epochs=run_config["epochs"],
    )

    print(model_config)
    print(trainer_config)

    dataset_manager = load_dataset_manager(
        dataset_name=trainer_config.dataset_name,
        dataset_dir=trainer_config.dataset_dir,
        data_dir="../data/",
        load_dataset=trainer_config.load_dataset,
        save_dataset=trainer_config.save_dataset,
        window_size=model_config.window_size,
    )
    print(
        "train:",
        len(dataset_manager.train_dataset),
        "valid:",
        len(dataset_manager.valid_dataset),
    )
    model = load_model(
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )

    def on_epoch_end(epoch: int):
        result = calc_accuracy(
            model=model,
            num_item=dataset_manager.num_item,
            test_dataset=dataset_manager.test_datasets["test"],
            top_k=[10, 20, 30, 40, 50],
        )
        print(epoch, result)
        results[run_config["name"]]["accuracy"].append(result)
        torch.save(model, f"cache/model/movielens-2/{run_config['name']}-{epoch}.pt")

    losses = trainer.fit(on_epoch_end=on_epoch_end)
    results[run_config["name"]]["loss"] = losses

    with open("result.json", "w") as f:
        json.dump(results, f)

ModelConfig(d_model=16, init_embedding_std=0.2, max_embedding_norm=1, window_size=5, negative_sample_size=5, lr=5e-05, weight_decay=1e-08, use_weight_tying=True, use_meta=True, use_attention=True)
TrainerConfig(model_name='doc2vec', dataset_name='movielens', epochs=10, batch_size=64, verbose=False, ignore_saved_model=True, load_model=False, save_model=True, load_dataset=False, save_dataset=False, model_dir='cache/model/', dataset_dir='cache/dataset/', device='cpu')
dataset_manager does not exist at: cache/dataset/movielens.pickle, create dataset
num_seq: 6040, num_item: 3706, num_item_meta: 28, num_seq_meta: 30, num_item_meta_types: 3, num_seq_meta_types: 3
to_sequential_data start
to_sequential_data end
train: 589192 valid: 110547


100%|██████████| 9207/9207 [00:34<00:00, 270.75it/s]


train 0.00649393937346458


100%|██████████| 1728/1728 [00:01<00:00, 1057.15it/s]


valid 0.006496575769230932


...

train 0.004648101541862188


100%|██████████| 1728/1728 [00:02<00:00, 845.04it/s]


valid 0.0047853116543910175


100%|██████████| 1591/1591 [00:08<00:00, 193.84it/s]

9 {'Accuracy@10': 0.13708359522313013, 'Accuracy@20': 0.13199245757385292, 'Accuracy@30': 0.12880787764508694, 'Accuracy@40': 0.12575424261470775, 'Accuracy@50': 0.12359522313010686}
saved model to cache/model/movielens/doc2vec.pt


In [7]:
top_k = list(range(10, 51, 10))
data = {}
for method, result in results.items():
    data[method] = []
    for k in top_k:
        a = max(map(lambda r: r[f"Accuracy@{k}"], result["accuracy"]))
        data[method].append(a)

In [10]:
df = pd.DataFrame(data).T
df.columns = [f"Accuracy@{k}" for k in top_k]
df

,Accuracy@10,Accuracy@20,Accuracy@30,Accuracy@40,Accuracy@50
User2Vec (d=16),0.186298,0.184381,0.180264,0.178158,0.175336
User2Vec (d=32),0.209805,0.204211,0.198617,0.192489,0.187404
User2Vec (d=64),0.211251,0.207102,0.202703,0.196229,0.191829
AU2V (d=16),0.215713,0.209585,0.206160,0.200629,0.195060
AU2V (d=32),0.226776,0.217788,0.209428,0.202263,0.195789
AU2V (d=64),0.229038,0.221150,0.212717,0.204824,0.198366
AU2V (wo weight-tying),0.226776,0.221307,0.210455,0.202923,0.197285
AU2V (wo attention),0.229101,0.221999,0.211544,0.202216,0.197260
AU2V (wo meta),0.227970,0.219233,0.209638,0.202043,0.195223
